# 2022年11月28日测试分析
连续10次测量长度为10米冷态钢轨

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

## 数据读取与平滑

In [12]:
# 预处理
def preprocess(data):
    hx = []
    h = []
    wx = []
    w = []
    for i in range(data.shape[1]):
        if data[1][i] > 0:
            hx.append(data[0][i])
            h.append(data[1][i])
        if data[2][i] > 0:
            wx.append(data[0][i])
            w.append(data[2][i])
    return np.asarray(hx), np.asarray(h), np.asarray(wx), np.asarray(w)

In [13]:
# 过滤异常值
def filter_outlier(data):
    data = np.nan_to_num(data)
    k = 5
    for i in range(k, data.shape[0] - k):
        temp = data[i - k : i + k].copy()
        temp[k] = (temp[k - 1] + temp[k + 1]) / 2
        data_mean = np.mean(temp)
        data_std = np.std(temp)
        left = data_mean - 1.5 * data_std
        right = data_mean + 1.5 * data_std
        if data[i] < left or data[i] > right:
            data[i] = data_mean
    return data

In [14]:
# 中值滤波
def filter_mid(data):
    data = np.nan_to_num(data)
    new_data = data.copy()
    k = 40
    i = 40
    data_mean = np.mean(data)
    dstd = np.std(data)
    print("data mean", data_mean, ' std ', dstd)
    while i < data.shape[0] - k:
        temp = data[i - k : i + k].copy()
        temp.sort()
        data_std = np.std(temp)
        for j in range(i - k, i + k):
            if np.abs(new_data[j] - data_mean) > dstd or j > data.shape[0] - 500:
                new_data[j] = data_mean
            elif np.abs(new_data[j] - temp[k]) > 1.5 * data_std:
                new_data[j] = temp[k]
        i += 2 * k
    temp = data[i - 2 * k : data.shape[0]].copy()
    temp.sort()
    data_std = np.std(temp)
    for ii in range(i - 2 * k, data.shape[0]):
        if np.abs(new_data[ii] - data_mean) > dstd or ii > data.shape[0] - 500:
            new_data[ii] = data_mean
    return new_data

In [19]:
# 均值滤波
def filter_mean(data):
    data = np.nan_to_num(data)
    new_data = data.copy()
    k = 50
    for i in range(k, data.shape[0] - k):
        temp = data[i - k : i + k]
        new_data[i] = np.mean(temp)
    return new_data

In [22]:
def filter(data):
    data = filter_mid(data)
    data = filter_mean(data)
    return data

In [23]:
all_data = pd.read_csv('correct_data.csv')
series = all_data['头宽1']

%matplotlib
plt.figure(figsize=(20, 10))
plt.subplot(2,1,1)
# 原始数据图
plt.plot(series)

# 滤波后
val = filter(series)
plt.subplot(2,1,2)
plt.plot(val)

Using matplotlib backend: QtAgg
data_mean 70.59030487684088  data std  8.754919815148515
